# This is a project for using the basic functionality of ifcopenshell

Main Challenge: 
**Filter out all load bearing walls and visualize them**

Optional:
**Pull other properties and dump it to a csv file.**

Main Challenge help: 
* A prerequisit for this challenge is the **01 Basic Introduction to Python for use with BIM notebook**
* The **02 Basics with ifcopenshell notebook** might also be of use. 

And like always in programming there is **google** and especially for opensource community work, like with ifcopenshell, you could ask the community on github. 

Note: In all cells below. Add code after the comments. 

In [1]:
# Importing relevant modules

import ifcopenshell
import ifcopenshell.geom
from ifc_viewer import ifc_viewer

## Main Challenge

### Create a function to get loarbaring walls and use it to get a list of loadbaring walls

In [3]:
# Create a function that takes in a tuple of IfcWall objects and returns the ones that are loadbaring in a list
# Use the "." operator to travers and get its property sets. It is useful to check the buildingSMART documentation on IfcWall
# Note: if you have a property in a variable p, then p.NominalValue.wrappedValue will give you the value.

def get_load_bearing(walls):
    def is_load_bearing(pset_wall_common):
        props = pset_wall_common.HasProperties
        for p in props:
            if p.Name =="LoadBearing":
                return p.NominalValue.wrappedValue
    # Thomas Krijens pro-tip: instead of appending to a list you could consider using a yield statement
    lb_walls = []
    for wall in walls:
        psets = [rel.RelatingPropertyDefinition for rel in wall.IsDefinedBy if rel.is_a("IfcRelDefinesByProperties")]
        pset_wall_commons = [pset for pset in psets if pset.Name == "Pset_WallCommon"]
        if len(pset_wall_commons) != 0:
            if is_load_bearing(pset_wall_commons[0]):
                lb_walls.append(wall)
        else:
            print("Wall %s has no Pset_WallCommon!!"%(wall.Name))
            
    return lb_walls

In [4]:
# Start with importing your chosen model. Store it in a variable. 
file = ifcopenshell.open("Grethes-hus-bok-2.ifc")

# Get a list of all walls and store it in a variable. 
walls = file.by_type("IfcWall")
#print(walls) 

# Use the function you have created, giving it the variable having the walls and store the result in another variable. 
load_bearing_walls = get_load_bearing(walls)

### Visualize 

Use some additional modules, like ```ifc_viewer``` to visualize the loadbaring walls below. 

In [6]:
## Set the proper geometry settings 
s = ifcopenshell.geom.settings()
s.set(s.USE_PYTHON_OPENCASCADE, True)

## Create a viewer object
viewer = ifc_viewer()

## Run trhough the loarbaring walls and create shapes and add that to the viewer object using the DisplayShape function. 
for wall in load_bearing_walls:
    shape = ifcopenshell.geom.create_shape(s, wall)
    viewer.DisplayShape(wall, shape.geometry, shape.styles)    

## Have the viewer display inline using the Display() function.
viewer.Display()


## Optional: Pull properties and dump it to a csv file.

In [7]:
# getting reference to all objects in the file that are of type "IfcWall"

walls = file.by_type("IfcWall")

print(f"There are {len(walls)} rooms in this model")

There are 24 rooms in this model


In [15]:
##Scroll through the walls and printing the value of your attributes

for wall in walls:                                        
    print(f"""
    Name:\t\t {wall.Name}
    Description: {wall.Description}\t\t
    GUID: {wall.GlobalId}""")


    Name:		 Basic Wall:Generic - 200mm:345653
    Description: None		
    GUID: 1xzRHg5wPCVvg4uLjqox1I

    Name:		 Basic Wall:Generic - 225mm Masonry:347110
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5cs

    Name:		 Basic Wall:Generic - 225mm Masonry:347115
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5cx

    Name:		 Basic Wall:Generic - 225mm Masonry:347120
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5cW

    Name:		 Basic Wall:Generic - 225mm Masonry:347132
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5ci

    Name:		 Basic Wall:Generic - 225mm Masonry:347296
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5xm

    Name:		 Basic Wall:Generic - 225mm Masonry:347303
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5xt

    Name:		 Basic Wall:Generic - 225mm Masonry:347308
    Description: None		
    GUID: 1Yzg4cwif7gQ2Hvbubk5xy

    Name:		 Basic Wall:_Bindeingsverk 316mm:348322
    Description: None		
    GUID: 3Q87Kkn4X0hh69O_upt9P1

  

In [19]:
# Write the GUID and the NAME of the walls in a csv file

import csv

with open('walls.csv', 'w') as csvfile:
    fieldnames = ['GUID', 'Name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for wall in walls:
        writer.writerow({'GUID': wall.GlobalId, 'Name': wall.Name})